Este script treina um modelo: LogisticRegression w/ BoW text preprocessing

Note que a acurácia deste modelo é a métrica de avaliação de modelo e portanto a principal métrica otimizada, que deve corresponder ao menos com o baseline de classe majoritária de 70% imposto pelo enunciado do EP.

Sugestões de trabalho e otimizações

- Normalizar os dados do treino [X_train], com cuidado para não normalizar os de teste [y_train], p/ evitar data leakage no treinamento.
- Melhorar pipeline de pre-processamento do BoW (Por exemplo, substituir BoW por TD-IDF, Ngrams, etc...)
- Calcular acurácia média do modelo. Pesquisar e introduzir novas métricas relevantes de avaliação com base no modelo e no problema.
- Estudar seleção de atributos de acordo com o modelo de LogisticRegression e aplicar p/ reduzir 
dimensionalidade dos dados (k-best, RFE, PCA, ExtraTrees, essas coisas...)
- Estudar otimizações p/ Hiperparâmetros
- Converter rotulos de strings p/ valores numéricos, afim de ser capaz de tirar a LogLoss do modelo.

In [ ]:
import pandas as pd
import numpy as np

Configura variáveis de execução

In [ ]:
sep = ";"
dec = ","
quotech = "\""
encoding = "utf8"


EP_dir = "EP1"
CSV_input_name = "train_complexo_simples.csv"
path_to_archive = f"../../../../Traindata/{EP_dir}/{CSV_input_name}"


do_print = True
if do_print:
    print(f"Path to csv input is:  {path_to_archive}")

Configure variáveis de reprodutibilidade

In [ ]:
random_state = 12345

Importar os dados do csv 

In [ ]:
df = pd.read_csv(path_to_archive, na_values=['na'],
sep=sep,
decimal=dec,
quotechar=quotech,
encoding=encoding,
encoding_errors='strict')
print(df.shape)
print(df.columns)

Embaralhamento dos dados

O .csv de entrada tem alto ordenamento dos inputs por classe. Carregá-los dessa maneira nos modelos p/ treinamento introduz viés, então é preciso embaralhar os dados para garantir randomicidade. 
As classes em sklearn.model_selection - como a StratfiedKFold usada mais a frente - implementam parâmetro shuffle="", que pode ser passado como True para embaralhar mais os dados.

Note que é importante também garantir a reprodutibilidade do embaralhamento, especificando um valor hardcoded (Neste caso random_state=100)

In [ ]:
print("Shape antes do shuffle:", df.shape)

df = df.sample(frac=1, random_state=100).reset_index(drop=True)

print("Shape depois do shuffle:", df.shape)



Prepara Bag of Words e rótulos

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

do_print = False

vect = CountVectorizer()
BoW_X = vect.fit_transform(df["text"].fillna("")) # features
y = df["style"].values # rótulos

print(BoW_X.shape)

Realiza treinamento c/ 10-fold cross validation do modelo 

Aqui - por enquanto - computamos também as métricas accuracy_score e log_loss. Pensei em computar também precision_score e recall_score, mas acredito que não faça sentido para o problema. Lembrando que:

- Accuracy: Precisão total do modelo. Computa o quanto o modelo acerta, em média, quando ele prevê tanto para negativo quanto para positivo. Mede a confiabilidade do modelo.
- Precision: Precisão positiva do modelo. Computa o quanto o modelo acerta quanto ele prevê positivo. Mede a confiabilidade das previsões positivas.
- Recall: Proporção de casos verdadeiro-positivos que o modelo acerta. Mede a capacidade de encontrar todos os positivos. Quando o valor é alto, significa que o modelo faz poucos falso-positivos.
- Log Loss: Métrica interessante do modelo LogisticRegression, que utiliza como Loss Function a tal da Log Loss. Indica para nós o quanto as probabilidades previstas pelo modelo estão bem calibradas. Valores baixos da LogLoss são melhores!

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, log_loss

# Definir StratifiedKFold
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=random_state)

accuracies = []
log_lossess = []

for fold, (train_idx, test_idx) in enumerate(skf.split(BoW_X, y), 1):
    print(f"Treinando fold {fold}...")
    X_train, X_test = BoW_X[train_idx], BoW_X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]

    clf = LogisticRegression(class_weight='balanced', max_iter=500).fit(X_train, y_train)
    predicted = clf.predict(X_test)
    accuracy = accuracy_score(y_test, predicted)
    #lloss = log_loss(y_test, predicted)
    
    accuracies.append(accuracy)
    #log_lossess.append(lloss)
    
    print("\n- LogisticRegression-BoW Acc =====>{:2.4f}".format(accuracy).replace(".",","))
    #print("    LogisticRegression-BoW LogLoss  =====>{:2.4f}\n\n".format(lloss).replace(".",","))


print(f"\n\nRaw Accuracies: {accuracies}")
#print(f"\nRaw Log Lossess: {log_lossess}")

Controle de reprodutibilidade

A sessão de código abaixo deve ser configurada com os parâmetros da solução ótima obtida através de experimentação, e executada somente para validar a reprodutibilidade da entrega.

Saída limpa significa que não houve quebra de reprodutibilidade com os resultados desenvolvidos ao longo do trabalho.

In [ ]:
import math

control_accuracies = [0.8130421776847143, 0.8232126832186659, 0.8120885697187313, 0.822262118491921, 0.8150807899461401, 0.8201675643327349, 0.8120885697187313, 0.8144823459006583, 0.801017354877319, 0.8195691202872532]
control_log_lossess = []

for idx, acc in enumerate(accuracies):
    if not math.isclose(acc, control_accuracies[idx], rel_tol=1e-11, abs_tol=1e-11):
        print(f"Quebra de controle in {idx}: Acc is {acc} and control Acc is {control_accuracies[idx]}")

for idx, lloss in enumerate(log_lossess):
    if not math.isclose(lloss, control_log_lossess[idx], rel_tol=1e-11, abs_tol=1e-11):
        print(f"Quebra de controle in {idx}: LogLoss is {lloss} and control LogLoss is {control_log_lossess[idx]}")